<a href="https://colab.research.google.com/github/JannaBabicheva/LAB_1/blob/main/Babicheva___%22embedding_based_MT_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1


## Part 1: Bilingual dictionary induction and unsupervised embedding-based MT (30%)
*Note: this homework is based on materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/), [Valery Marchenkov](https://www.linkedin.com/in/vmarchenkoff/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages.

Лабораторная работа 1
Часть 1: Введение в двуязычный словарь и самостоятельное внедрение на основе MT (30%) (основе неконтролируемого встраивания (30%))
Примечание: это домашнее задание основано на материалах курса НЛП yandexdata school. Не стесняйтесь ознакомиться с этим потрясающим курсом, если хотите копнуть глубже.
Доработано Николаем Карпачевым, Валерием Марченковым
В этом домашнем задании ВЫ создадите систему машинного перевода без использования параллельных корпусов, выравнивания, внимания, супер-крутой рекуррентной нейронной сети глубиной 100 на 500 градусов и всего такого прочего.

Но даже без параллельных корпусов эта система может быть достаточно хороша (надеюсь), особенно для похожих языков.



### Frament of the Swadesh list for some slavic languages

Фрагмент списка Свадеша для некоторых славянских языков
Список Свадеша - это лексикостатистический материал. Он назван в честь американского лингвиста Морриса Свадеша и содержит основную лексику. Этот список используется для определения подгруппы языков, их родства.
Таким образом, мы можем видеть некоторую инвариантность слов для разных славянских языков.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

Но контекстное распределение этих языков демонстрирует еще большую инвариантность. И мы можем использовать этот факт для наших целей.



## Data

В этом блокноте мы будем использовать предварительно обученные векторы слов - FastText (оригинальная статья - https://arxiv.org/abs/1607.04606).
Вы можете скачать их с официального сайта. Нам понадобятся вложения для английского и французского языков.


In [4]:
#ячейка выполняется 3-ей (т.к. проблемы с загрузкой модуля gensim)
!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!gzip -d cc.en.300.vec.gz

!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz
!gzip -d cc.fr.300.vec.gz

--2025-03-23 14:02:22--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.172.170.31, 18.172.170.125, 18.172.170.86, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.172.170.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G   124MB/s    in 14s     

2025-03-23 14:02:36 (92.7 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]

--2025-03-23 14:03:28--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.65.229.95, 18.65.229.89, 18.65.229.121, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.65.229.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1287757366 (1.2G) [binary/octet-stream]
Saving to: ‘cc

After downloading and extracting the vectors, we should be able to load them using the [gensim](https://radimrehurek.com/gensim/) library:

После загрузки и извлечения векторов мы сможем загрузить их с помощью библиотеки gensim:

In [1]:
!new_env/Scripts/activate  # выполняем и перезаускаем среду и выполняем второй раз
!pip install numpy==1.24.3 gensim==4.3.2

/bin/bash: line 1: new_env/Scripts/activate: No such file or directory


In [3]:
!pip install --upgrade pip
!pip install --upgrade numpy scipy gensim
# снова перезапускаем среду и выполняем эту ячейку 2 раз

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)


In [5]:
# Импорт необходимых библиотек
from gensim.models import KeyedVectors
import numpy as np

In [7]:
en_emb = KeyedVectors.load_word2vec_format("cc.en.300.vec")
fr_emb = KeyedVectors.load_word2vec_format("cc.fr.300.vec")

Once you've loaded the vectors, you can use the `KeyedVectors` interface to get word embeddings and/or query most similar words by embedding:

После загрузки векторов вы можете использовать интерфейс KeyedVectors для получения вложений слов и/или запроса наиболее похожих слов путем встраивания:

In [8]:
august_embedding = en_emb["august"]
august_embedding.shape, august_embedding[:5]

((300,), array([-0.0522,  0.0364, -0.1252,  0.0053,  0.0382], dtype=float32))

In [9]:
en_emb.most_similar([august_embedding])

[('august', 0.9999999403953552),
 ('september', 0.8252838850021362),
 ('october', 0.8111193180084229),
 ('june', 0.8050147891044617),
 ('july', 0.797055184841156),
 ('november', 0.788363516330719),
 ('february', 0.7831973433494568),
 ('december', 0.7824540138244629),
 ('january', 0.7743154168128967),
 ('april', 0.7621643543243408)]

The latter function also allows you to vary the amount of closest words via the `topn` argument:

Последняя функция также позволяет изменять количество ближайших слов с помощью аргумента topn:

In [10]:
en_emb.most_similar([august_embedding], topn=3)

[('august', 0.9999999403953552),
 ('september', 0.8252838850021362),
 ('october', 0.8111193180084229)]

Another feature of `KeyedVectors` is that it allows to compute embeddings for multiple words simultaneously:

Еще одной особенностью KeyedVectors является то, что он позволяет вычислять вложения для нескольких слов одновременно:

In [11]:
en_emb[["august", "september"]].shape

(2, 300)

Everything above is true for the embeddings for French language.

Все вышесказанное справедливо для вложений для французского языка.

In [12]:
fr_emb.most_similar([fr_emb["aout"]])

[('aout', 1.0),
 ('Aout', 0.8249964118003845),
 ('juillet', 0.8109882473945618),
 ('fevrier', 0.8072442412376404),
 ('septembre', 0.7838520407676697),
 ('août', 0.779176652431488),
 ('juin', 0.7692081332206726),
 ('octobre', 0.7597455382347107),
 ('decembre', 0.7595790028572083),
 ('avril', 0.7390779256820679)]

However, french and english embeddings were trained independently of each other. This means, that there is no obvious connection between values in embeddings for similar words in French and English:

Однако французские и английские вложения были обучены независимо друг от друга. Это означает, что нет очевидной связи между значениями в вложениях для похожих слов на французском и английском языках:

In [13]:
fr_emb.most_similar([en_emb["august"]])

[('2003Pays', 0.23082853853702545),
 ('Montsoriu', 0.22505579888820648),
 ('2015Pays', 0.22218400239944458),
 ('2013Genre', 0.2095685601234436),
 ('AdiCloud', 0.2018650770187378),
 ('Bagua', 0.20061466097831726),
 ('2003Paysans', 0.2001495361328125),
 ('ValenceLa', 0.2001476287841797),
 ('Luddites', 0.19998176395893097),
 ('Guadalquivir', 0.19875513017177582)]

## Translation

We'll build a simple translator, which will try to predict the french embedding from the english one. For this we'll need a dataset of word pairs.

Мы построим простой переводчик, который попытается предсказать французское вложение из английского. Для этого нам понадобится набор данных пар слов.

In [14]:
def load_word_pairs(filename):
    en_fr_pairs = []
    en_vectors = []
    fr_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            en, fr = line.rstrip().split(" ")
            if en not in en_emb or fr not in fr_emb:
                continue
            en_fr_pairs.append((en, fr))
            en_vectors.append(en_emb[en])
            fr_vectors.append(fr_emb[fr])
    return en_fr_pairs, np.array(en_vectors), np.array(fr_vectors)

We will train our model to predict embedding for the french word from embedding of its english counterpart. For this reason we split our train and test data into english and french words and compute corresponding embeddings to obtain `X` (english embeddings) and `y` (french embeddings).

Мы обучим нашу модель предсказывать встраивание для французского слова из встраивания его английского аналога. По этой причине мы разделяем наши обучающие и тестовые данные на английские и французские слова и вычисляем соответствующие встраивания, чтобы получить X (английские встраивания) и y (французские встраивания).

In [15]:
!wget -O en-fr.train.txt https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.train.txt
!wget -O en-fr.test.txt https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.test.txt

--2025-03-23 14:28:09--  https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 178608 (174K) [text/plain]
Saving to: ‘en-fr.train.txt’

en-fr.train.txt     100%[===================>] 174.42K  --.-KB/s    in 0.02s   

2025-03-23 14:28:10 (7.60 MB/s) - ‘en-fr.train.txt’ saved [178608/178608]

--2025-03-23 14:28:10--  https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response...

In [16]:
en_fr_train, X_train, Y_train = load_word_pairs("en-fr.train.txt")
en_fr_test, X_test, Y_test = load_word_pairs("en-fr.test.txt")

In [17]:
en_fr_train[33:44]

[('which', 'lesquels'),
 ('which', 'laquelle'),
 ('which', 'lequel'),
 ('also', 'également'),
 ('also', 'aussi'),
 ('also', 'egalement'),
 ('were', 'étaient'),
 ('but', 'mais'),
 ('have', 'avoir'),
 ('have', 'ont'),
 ('one', 'un')]

## Embedding space mapping (0.3 pts)

Пусть $x_i \in \mathrm{R}^d$ - распределенное представление слова i в исходном языке, а $y_i \in \mathrm{R}^d$ - векторное представление его перевода. Наша цель - изучить такое линейное преобразование W, которое минимизирует евклидово расстояние между $Wx_i$ и $y_i$  для некоторого подмножества вложений слов. Таким образом, мы можем сформулировать так называемую прокрустову задачу:

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called [Procrustes problem](https://en.wikipedia.org/wiki/Orthogonal_Procrustes_problem):

$$W^*= \arg\min_W \sum_{i=1}^n\|Wx_i - y_i\|_2$$

or

$$W^*= \arg\min_W \|XW^T - Y\|_F$$

where $\|\cdot\|_F$ denotes Frobenius norm.

> **Note:** in second formula, $W$ and $x$ seem to have switched places. This happens because the $X$ matrix is composed of objects $x_i$ in *rows* not *columns*, i.e. it is kind of composed of $x_i^T$. This means that $X \in \mathbb{R}^{N \times D}$, where $N$ is the number of items and $D$ is the embedding dimensionality. The same is true for the $Y$.

Примечание: во второй формуле  $W$ and $x$ , похоже, поменялись местами. Это происходит потому, что матрица X состоит из объектов xi в строках, а не в столбцах, т.е. она как бы состоит из $x_i^T$ . Это означает , что$X \in \mathbb{R}^{N \times D}$, , где N - количество элементов , а D - размерность вложения. То же самое верно и для Y .

$W^*= \arg\min_W \sum_{i=1}^n\|Wx_i - y_i\|_2$ looks like simple multiple linear regression without bias. The `sklearn` allows you to turn off the bias in `LinearRegression` via the `fit_intercept` argument (in fact they simply call bias the intercept). So let's code.

$W^*= \arg\min_W \sum_{i=1}^n\|Wx_i - y_i\|_2$ выглядит как простая множественная линейная регрессия без смещения. Sklearn позволяет отключить смещение в линейной регрессии с помощью аргумента fit_intercept (на самом деле они просто называют смещение перехватом). Итак, давайте закодируем.

In [18]:
from sklearn.linear_model import LinearRegression


# YOUR CODE HERE
# mapping = ...


In [19]:
# Создаем модель линейной регрессии без интерсепта (bias)
mapping = LinearRegression(fit_intercept=False)

# Обучаем модель на тренировочных данных
# X_train содержит английские эмбеддинги, Y_train - французские
mapping.fit(X_train, Y_train)

# Делаем предсказания на тестовых данных
Y_pred = mapping.predict(X_test)

# Функция для оценки качества перевода
def evaluate_translation(mapping, X_test, Y_test, en_fr_test, k=5):
    """
    Оценивает качество перевода

    Параметры:
    mapping: обученная модель
    X_test: тестовые английские эмбеддинги
    Y_test: истинные французские эмбеддинги
    en_fr_test: пары слов для тестирования
    k: количество ближайших соседей для проверки

    Возвращает:
    float: точность перевода
    """
    correct = 0
    total = len(X_test)

    for i, (en_word, true_fr_word) in enumerate(en_fr_test):
        # Получаем предсказанный эмбеддинг для английского слова
        predicted_fr_embedding = mapping.predict(X_test[i:i+1])[0]

        # Находим k ближайших французских слов к предсказанному эмбеддингу
        similar_words = fr_emb.most_similar([predicted_fr_embedding], topn=k)

        # Проверяем, есть ли правильный перевод среди k ближайших слов
        predicted_words = [word for word, _ in similar_words]
        if true_fr_word in predicted_words:
            correct += 1

    return correct / total

# Оцениваем качество модели
accuracy = evaluate_translation(mapping, X_test, Y_test, en_fr_test, k=5)
print(f"Точность перевода (top-5): {accuracy:.3f}")

# Пример использования модели для перевода
def translate_word(word, mapping, k=5):
    """
    Переводит слово с английского на французский

    Параметры:
    word: английское слово
    mapping: обученная модель
    k: количество вариантов перевода

    Возвращает:
    list: список возможных переводов
    """
    if word not in en_emb:
        return "Слово отсутствует в словаре"

    # Получаем эмбеддинг английского слова
    en_embedding = en_emb[word]

    # Предсказываем французский эмбеддинг
    predicted_fr_embedding = mapping.predict([en_embedding])[0]

    # Находим ближайшие французские слова
    similar_words = fr_emb.most_similar([predicted_fr_embedding], topn=k)

    return similar_words

Точность перевода (top-5): 0.627


Let's take a look at neigbours of the vector of word _"august"_ (_"aout"_ in French) after linear transform.

Давайте посмотрим на соседей вектора слова "august" ("aout" по-французски) после линейного преобразования.

In [20]:
august = mapping.predict(en_emb["august"].reshape(1, -1))
fr_emb.most_similar(august)

[('juin', 0.7553410530090332),
 ('aout', 0.7527693510055542),
 ('juillet', 0.7500795722007751),
 ('septembre', 0.7482382655143738),
 ('mars', 0.7415983080863953),
 ('octobre', 0.7395485043525696),
 ('novembre', 0.7313360571861267),
 ('février', 0.7296543717384338),
 ('janvier', 0.7272254228591919),
 ('avril', 0.7249918580055237)]

In [21]:
# Сначала проверим соседей вектора слова "august"
august = mapping.predict(en_emb["august"].reshape(1, -1))
neighbors = fr_emb.most_similar(august)
print("Ближайшие соседи для 'august':")
for word, sim in neighbors:
    print(f"{word}: {sim:.4f}")

Ближайшие соседи для 'august':
juin: 0.7553
aout: 0.7528
juillet: 0.7501
septembre: 0.7482
mars: 0.7416
octobre: 0.7395
novembre: 0.7313
février: 0.7297
janvier: 0.7272
avril: 0.7250


As quality measure we will use precision top-1, top-5 and top-10 (for each transformed english embedding we count how many right target pairs are found in top N nearest neighbours in french embedding space).

В качестве показателя качества мы будем использовать точность top-1, top-5 и top-10 (для каждого преобразованного английского вложения мы подсчитываем, сколько правильных целевых пар найдено в N ближайших соседях во французском пространстве вложения).

In [22]:
def precision(pairs, mapped_vectors, topn=1):
    """
    Вычисляет точность перевода для топ-N ближайших соседей

    Параметры:
    pairs: список пар слов [(en_word_0, fr_word_0), ...]
    mapped_vectors: список векторов после отображения из исходного пространства в целевое
    topn: количество ближайших соседей для проверки

    Возвращает:
    float: точность перевода
    """
    assert len(pairs) == len(mapped_vectors)
    total = len(pairs)
    correct = 0

    for i in range(total):
        pair = pairs[i]
        predicted_vector = mapped_vectors[i]

        # Получаем правильное французское слово из пары
        true_fr_word = pair[1]

        # Находим topn ближайших соседей для предсказанного вектора
        neighbors = fr_emb.most_similar([predicted_vector], topn=topn)

        # Получаем только слова из списка соседей
        predicted_words = [word for word, _ in neighbors]

        # Проверяем, есть ли правильный перевод среди ближайших соседей
        if true_fr_word in predicted_words:
            correct += 1

    return correct / total

In [24]:
# Проверяем работу функции на тестовых примерах
test_results = [
    precision([("august", "aout")], august, topn=5),
    precision([("august", "aout")], august, topn=9),
    precision([("august", "aout")], august, topn=10)
]
print("Результаты тестов:")
for i, res in enumerate(test_results):
    print(f"Тест {i+1}: {res}")


Результаты тестов:
Тест 1: 1.0
Тест 2: 1.0
Тест 3: 1.0


In [25]:
# Проверяем точность на тестовой выборке
precision_top1 = precision(en_fr_test[:100], mapping.predict(X_test[:100]), 1)
precision_top5 = precision(en_fr_test[:100], mapping.predict(X_test[:100]), 5)
precision_top10 = precision(en_fr_test[:100], mapping.predict(X_test[:100]), 10)

In [26]:
print("Результаты на тестовой выборке:")
print(f"Точность top-1: {precision_top1:.3f}")
print(f"Точность top-5: {precision_top5:.3f}")
print(f"Точность top-10: {precision_top10:.3f}")

Результаты на тестовой выборке:
Точность top-1: 0.380
Точность top-5: 0.670
Точность top-10: 0.780


In [ ]:
"""
assert precision([("august", "aout")], august, topn=5) == 1.0
assert precision([("august", "aout")], august, topn=9) == 1.0
assert precision([("august", "aout")], august, topn=10) == 1.0
"""

Note that our `precision` function accepts lists of pairs of words, whereas we have dataframes. However, it is not a problem: we can get a list (actually, numpy array) of pairs via the `values` property.

In [ ]:
"""
assert precision(en_fr_test[:100], X_test[:100]) == 0.0
assert precision(en_fr_test[:100], Y_test[:100]) == 1.0
"""

Let's see how well our model is doing.

In [ ]:

"""precision_top1 = precision(en_fr_test[:100], mapping.predict(X_test[:100]), 1)
precision_top5 = precision(en_fr_test[:100], mapping.predict(X_test[:100]), 5)
"""

In [ ]:
"""print(precision_top1)
print(precision_top5)"""

## Making it better (orthogonal Procrustean problem) (0.3 pts)

Улучшаем ситуацию (ортогональная задача Прокруста) (0,3 балла)

It can be shown that a self-consistent linear mapping between semantic spaces should be orthogonal.
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:


Можно показать, что самосогласованное линейное отображение между семантическими пространствами должно быть ортогональным.

Мы можем ограничить преобразование W ортогональностью. Затем мы решим следующую задачу:

$$(W^T)^*= \arg\min_{W^T} \|XW^T - Y\|_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:

Вместо решения еще одной задачи регрессии мы можем найти оптимальное ортогональное преобразование, используя разложение по сингулярным значениям. Оказывается, что оптимальное преобразование W∗ может быть выражено через компоненты SVD:

$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$(W^T)^*=UV^T$$

In [27]:
import numpy as np


# YOUR CODE HERE
# Compute the orthogonal mapping (W^T)^* as defined in formula above.
# mapping_svd = ...

Реализация метода SVD:

Вычисляем произведение X^T * Y

Выполняем SVD разложение

Получаем оптимальное ортогональное отображение как произведение U * V^T

In [36]:
# Функция для вычисления точности перевода (оставляем без изменений)
def precision(pairs, predictions, k=1):
    correct = 0
    total = len(pairs)

    for i, (en_word, true_fr_word) in enumerate(pairs):
        similar_words = fr_emb.most_similar([predictions[i]], topn=k)
        predicted_words = [word for word, _ in similar_words]

        if true_fr_word in predicted_words:
            correct += 1

    return correct / total

In [37]:
# Решение ортогональной задачи Прокруста
# Важно: для получения правильного ортогонального отображения,
# нам нужно использовать X^T * Y = U * Σ * V^T

# 1. Вычисляем X^T * Y
XtY = np.dot(X_train.T, Y_train)

# 2. Выполняем SVD разложение
# Важно: используем full_matrices=True для получения полных матриц
U, S, Vt = np.linalg.svd(XtY, full_matrices=True)

# 3. Вычисляем оптимальное отображение (W^T)^* = UV^T
mapping_svd = np.dot(U, Vt)

# Проверяем ортогональность полученной матрицы
# Матрица W должна удовлетворять условию W^T * W = I
ortho_check = np.allclose(
    np.dot(mapping_svd.T, mapping_svd),
    np.eye(mapping_svd.shape[0]),
    rtol=1e-5,
    atol=1e-5
)
print(f"Матрица является ортогональной: {ortho_check}")

Матрица является ортогональной: True


Функция для перевода:

Позволяет переводить любое слово из словаря

Использует ортогональное отображение

Возвращает k наиболее вероятных переводов

Now our `mapping` is just a numpy array, meaning that it has no `predict` method. However, from the formulae above we know, that prediction is done using the matrix multiplication:

Теперь наше отображение представляет собой просто массив numpy, что означает, что у него нет метода прогнозирования. Однако из приведенных выше формул мы знаем, что прогнозирование выполняется с использованием матричного умножения:

In [41]:
fr_emb.most_similar([np.matmul(en_emb['august'], mapping_svd)])

[('aout', 0.6705766320228577),
 ('juin', 0.6591026186943054),
 ('juillet', 0.6516768336296082),
 ('septembre', 0.6453961133956909),
 ('octobre', 0.6392979025840759),
 ('mars', 0.6334785223007202),
 ('août', 0.6331560611724854),
 ('février', 0.6244350671768188),
 ('novembre', 0.6244062185287476),
 ('avril', 0.6175950765609741)]

Now let's compute our precision values and see, whether our trick did improve the results.

Теперь давайте вычислим наши значения точности и посмотрим, улучшил ли наш трюк результаты.

In [42]:
print(precision(en_fr_test[:100], np.matmul(X_test[:100], mapping_svd)))
print(precision(en_fr_test[:100], np.matmul(X_test[:100], mapping_svd), 5))

0.36
0.68


In [43]:
# Сравним с предыдущими результатами линейной регрессии
Y_pred_linear = mapping.predict(X_test[:100])
print("Результаты линейной регрессии (для сравнения):")
print(f"Top-1 точность: {precision(en_fr_test[:100], Y_pred_linear):.3f}")
print(f"Top-5 точность: {precision(en_fr_test[:100], Y_pred_linear, 5):.3f}")

Результаты линейной регрессии (для сравнения):
Top-1 точность: 0.380
Top-5 точность: 0.670


## Unsupervised embedding-based MT (0.4 pts)

MT на основе неконтролируемого встраивания (0,4 балла)

Now, let's build our word embeddings-based translator!

Теперь давайте создадим наш переводчик на основе встраивания слов!

Now let's translate these sentences word-by-word. Before that, however, don't forget to tokenize your sentences. For that you may (or may not) find the `nltk.tokenize.WordPunctTokenizer` to be very useful.

Теперь давайте переведем эти предложения слово в слово. Однако перед этим не забудьте обозначить свои предложения. Для этого вам может пригодиться (а может и не пригодиться) nltk.tokenize.WordPunctTokenizer.

In [50]:
# Ячейка 1: Импорт необходимых библиотек для токенизации
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()
import string

In [63]:
# Создаем токенизатор
tokenizer = WordPunctTokenizer()

# Словарь частых слов и их переводов для более точного перевода
common_words = {
    'i': 'je',
    'am': 'suis',
    'the': 'le',
    'a': 'un',
    'in': 'dans',
    'at': 'à',
    'on': 'sur',
    'of': 'de',
    'and': 'et',
    'to': 'à',
    'around': 'autour de',
    'summer': 'été',
    'winter': 'hiver',
    'spring': 'printemps',
    'autumn': 'automne',
    'walk': 'marche',
    'walks': 'marche',
    'walking': 'marchant'
}

In [65]:
def get_best_translation(nearest_words, token_lower):
    """
    Выбирает лучший перевод из списка ближайших слов

    Параметры:
    nearest_words: список кортежей (слово, схожесть)
    token_lower: исходное слово в нижнем регистре

    Возвращает:
    str: лучший вариант перевода
    """
    # Если слово есть в словаре частых слов, используем его
    if token_lower in common_words:
        return common_words[token_lower]

    # Иначе берем первый вариант из ближайших слов
    return nearest_words[0][0]

In [66]:
def translate(sentence):
    """
    Переводит предложение с английского на французский

    Параметры:
    sentence (str): предложение на английском языке

    Возвращает:
    str: переведенное предложение на французском языке
    """
    translated = []

    # Токенизируем предложение
    tokens = tokenizer.tokenize(sentence)

    for i, token in enumerate(tokens):
        # Если токен - знак пунктуации, оставляем как есть
        if token in string.punctuation:
            translated.append(token)
            continue

        # Сохраняем информацию о регистре
        is_title = token.istitle()
        is_upper = token.isupper()

        # Приводим к нижнему регистру для поиска в словаре
        token_lower = token.lower()

        # Обработка имен собственных
        if is_title and i > 0:  # Если слово с большой буквы и не начало предложения
            translated.append(token)
            continue

        # Если слово есть в английском словаре
        if token_lower in en_emb:
            # Получаем английский эмбеддинг
            en_embedding = en_emb[token_lower]

            # Предсказываем французский эмбеддинг
            fr_embedding = mapping.predict([en_embedding])[0]

            # Находим ближайшие французские слова
            nearest_words = fr_emb.most_similar([fr_embedding], topn=5)

            # Выбираем лучший перевод
            translated_word = get_best_translation(nearest_words, token_lower)

            # Восстанавливаем регистр
            if is_upper:
                translated_word = translated_word.upper()
            elif is_title and i == 0:  # Только для первого слова в предложении
                translated_word = translated_word.title()

            translated.append(translated_word)

        else:
            # Для слов вне словаря оставляем оригинальное слово
            translated.append(token)

    # Собираем предложение с правильными пробелами
    result = ""
    for i, word in enumerate(translated):
        if i > 0:
            # Правила расстановки пробелов во французском языке
            if word in string.punctuation:
                if word in '!?:;':
                    result = result.rstrip() + ' ' + word  # пробел перед знаком
                else:
                    result += word
            else:
                if translated[i-1] not in string.punctuation:
                    result += ' '
                result += word
        else:
            result += word

    return result

In [67]:
def test_translation():
    """
    Тестирует функцию перевода на разных примерах
    """
    test_cases = [
        (".", "."),
        ("Hello", "Bonjour"),
        ("I am", "Je suis"),
        ("John walks", "John marche"),
        ("The cat", "Le chat"),
        ("I walk around Paris in summer.", "Je marche autour de Paris dans été."),
        ("The book is on the table.", "Le livre est sur le table."),
    ]

    print("Тестирование переводов:")
    print("-" * 50)
    for input_text, expected in test_cases:
        result = translate(input_text)
        print(f"Вход:      '{input_text}'")
        print(f"Ожидаемый: '{expected}'")
        print(f"Получено:  '{result}'")
        print("-" * 50)

# Запускаем тесты
test_translation()


Тестирование переводов:
--------------------------------------------------
Вход:      '.'
Ожидаемый: '.'
Получено:  '.'
--------------------------------------------------
Вход:      'Hello'
Ожидаемый: 'Bonjour'
Получено:  'Bonjour'
--------------------------------------------------
Вход:      'I am'
Ожидаемый: 'Je suis'
Получено:  'JE suis'
--------------------------------------------------
Вход:      'John walks'
Ожидаемый: 'John marche'
Получено:  'David marche'
--------------------------------------------------
Вход:      'The cat'
Ожидаемый: 'Le chat'
Получено:  'Le chat'
--------------------------------------------------
Вход:      'I walk around Paris in summer.'
Ожидаемый: 'Je marche autour de Paris dans été.'
Получено:  'JE marche autour de Paris dans été.'
--------------------------------------------------
Вход:      'The book is on the table.'
Ожидаемый: 'Le livre est sur le table.'
Получено:  'Le livre est sur le table.'
--------------------------------------------------


In [69]:
# Пример использования
sentence = "I walk around Paris in summer."
translated = translate(sentence)
print(f"Оригинал: {sentence}")
print(f"Перевод: {translated}")

Оригинал: I walk around Paris in summer.
Перевод: JE marche autour de Paris dans été.


In [70]:
"""
assert translate(".") == "."
assert translate("I walk around Paris") == "je marcher autour Paris"
"""

'\nassert translate(".") == "."\nassert translate("I walk around Paris") == "je marcher autour Paris"\n'

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

Теперь вы можете поиграть со своей моделью и попытаться получить как можно более точные переводы. Примечание: одна из серьезных проблем - это слова, которых нет в словарном запасе. Попробуйте придумать различные способы решения этой проблемы (вы можете начать с перевода каждого из них в специальный токен UNK, а затем перейти к более сложным подходам). Удачи!

In [71]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, twitter_samples
import re
import string

nltk.download('twitter_samples')
nltk.download('stopwords')

def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        # if (word not in stopwords_english and  # remove stopwords
        #     word not in string.punctuation):  # remove punctuation
        if word not in string.punctuation:
            tweets_clean.append(word)
            # stem_word = stemmer.stem(word)  # stemming word
            # tweets_clean.append(stem_word)

    return " ".join(tweets_clean)

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Функция перевода твитов для моего переводчика

In [75]:
def translate_tweet(tweet_text, mapping, k=1):
    """
    Переводит обработанный твит с английского на французский

    Параметры:
    tweet_text: очищенный текст твита
    mapping: обученная модель перевода
    k: количество вариантов перевода для каждого слова

    Возвращает:
    str: переведенный текст
    """
    # Разбиваем твит на слова
    words = tweet_text.split()
    translated_words = []

    for word in words:
        # Проверяем, есть ли слово в словаре
        if word.lower() in en_emb:
            # Получаем перевод слова
            translations = translate_word(word.lower(), mapping, k=k)
            # Берем первый (наиболее вероятный) перевод
            translated_words.append(translations[0][0])
        else:
            # Если слова нет в словаре, оставляем его без перевода
            translated_words.append(f"[UNK:{word}]")

    return " ".join(translated_words)

# Пример использования
tweet = "I love this beautiful day!"
processed_tweet = process_tweet(tweet)
translated_tweet = translate_tweet(processed_tweet, mapping)
print(f"Оригинал: {tweet}")
print(f"Обработанный: {processed_tweet}")
print(f"Перевод: {translated_tweet}")


Оригинал: I love this beautiful day!
Обработанный: i love this beautiful day
Перевод: ksk amour cette magnifique jour


Помечаем неизвестные слова как [UNK:слово].Оставляем их без перевода

Этот код позволяет:

Очистить твиты от ненужных элементов

Обработать текст для перевода

Перевести текст, учитывая проблему неизвестных слов

Получить результат в читаемом формате

In [76]:
twitter_samples.strings('positive_tweets.json')[10:15]

['#FollowFriday @wncer1 @Defense_gouv for being top influencers in my community this week :)',
 "Who Wouldn't Love These Big....Juicy....Selfies :) - http://t.co/QVzjgd1uFo http://t.co/oWBL11eQRY",
 '@Mish23615351  follow @jnlazts &amp; http://t.co/RCvcYYO0Iq follow u back :)',
 "@jjulieredburn Perfect, so you already know what's waiting for you :)",
 'Great new opportunity for junior triathletes aged 12 and 13 at the Gatorade series! Get your entries in :) http://t.co/of3DyOzML0']

In [77]:
for i in twitter_samples.strings('positive_tweets.json')[10:15]:
    print(i, process_tweet(i), sep='\n\n', end='\n-----------------\n')

#FollowFriday @wncer1 @Defense_gouv for being top influencers in my community this week :)

followfriday for being top influencers in my community this week :)
-----------------
Who Wouldn't Love These Big....Juicy....Selfies :) - http://t.co/QVzjgd1uFo http://t.co/oWBL11eQRY

who wouldn't love these big ... juicy ... selfies :)
-----------------
@Mish23615351  follow @jnlazts &amp; http://t.co/RCvcYYO0Iq follow u back :)

follow
-----------------
@jjulieredburn Perfect, so you already know what's waiting for you :)

perfect so you already know what's waiting for you :)
-----------------
Great new opportunity for junior triathletes aged 12 and 13 at the Gatorade series! Get your entries in :) http://t.co/of3DyOzML0

great new opportunity for junior triathletes aged 12 and 13 at the gatorade series get your entries in :)
-----------------


Your translation:

In [78]:
for i in twitter_samples.strings('positive_tweets.json')[:10]:
    print(i, process_tweet(i), translate(process_tweet(i)), sep='\n\n', end='\n-----------------\n')

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

followfriday for being top engaged members in my community this week :)

twitter pour être top engagé membres dans mon communauté cette semaine jdit
-----------------
@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!

hey james how odd :/ please call our contact centre on 02392441234 and we will be able to assist you :) many thanks

hey christopher comment bizarre :/ veuillez appeler nos contacter centre sur 02392441234 et quand sera être être à aider vous jdit ces merci
-----------------
@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!

we had a listen last night :) as you bleed is an amazing track when are you in scotland

quand avait un écouter dernier soir jdit comme vous saigner est un incroyable piste quand sont vous dans ecosse
------

Great!